# Developing and Comparing Sequential and Distributed Algorithms with maia

## Introduction

Using **Maia**, a Python/C++ library for working with CGNS meshes in parallel with MPI. The idea of this activity is to write an algorithm in a "distributed way", ie. operating on a distributed tree.

The objective is to compare three methods for calculating the geometric centers of mesh cells from a CGNS tree:
1. A sequential algorithm,
2. A parallel algorithm on a partitioned tree,
3. A fully distributed algorithm.

You'll discover how to: 
- Load CGNS trees in distributed, sequential, partitioned version and compare the execution times,
- Apply field calculations across unstructured meshes,
- Utilize Maia's exchange and indexing tools to perform tasks in parallel,
- Examine the trade-offs between each method's design and performance.

These exercises, which are particularly appropriate for individuals wishing to expand or enhance parallel mesh processing pipelines, demonstrates a common development task when developing new functionalities in Maia.

## Step 1 -- import modules

Maia operates in parallel! The so-called COMM_WORLD communicator must be imported from mpi4py first because practically all functions require an MPI communicator.
We need to import COMM_WORLD communicator from mpi4py, numpy, and time.

In [1]:
import time
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD

Open the documentation that will be useful for this TP first: /Maia/1.3/index.html https://onera.github.io.
Take a brief look at the structure of the various modules (UserManual) and the definition of parallel CGNSTree (Introducion > Maia CGNS Tree). Then, import maia and the module pytree of maia. 

In [2]:
import maia
import maia.pytree as PT

In [3]:
FILENAME = '/home/jcoulet/Public/maia_training/MESHES/tetra10.hdf'

In [4]:
def _generate_case():
    from pathlib import Path
    if not Path(FILENAME).exists():
        tree = maia.factory.generate_dist_block(11, 'TETRA_4', comm)
        maia.io.dist_tree_to_file(tree, FILENAME, comm)

## Step 2 -- Create an instance

- Create an instance of the DIndexer class by providing:

    - a distribution tree (distri),

    - a list of indices (indices),

    - and a communicator (comm, e.g., from MPI).

- Use the take(data_in) method to extract and transfer data based on the given indices and the parallel distribution.

- The method returns the portion of the data that corresponds to the local partition.

In [5]:
class DIndexer:
    def __init__(self, distri, indices, comm):
        from maia.transfer import protocols as EP
        self.btp = EP.GlobalIndexer(distri, indices-1, comm)

    def take(self, data_in):
        return self.btp.Take(data_in)


## Step 3 -- Calcul cells's center withn a CGNS zone

- Create a function called compute_cc_seq(tree) that determines each cell's geometric center, or centroid, within a CGNS zone.

- Within the function:

    - Use PT.get_node_from_label(tree, 'Zone_t') to retrieve the zone node from the tree.

    - Use PT.Zone.coordinates(zone) to extract the node coordinates cx, cy, and cz.

    - Use PT.get_node_from_name(zone, 'ElementConnectivity')[1] to obtain the cell connectivity array.

- Assume that every cell is a quadruple consisting of four nodes.

    - Use connec_idx = 4 * np.arange(n_elem+1) to calculate the number of elements and construct the connectivity index array.

- To determine the average coordinates (mean_x, mean_y, and mean_z) for every cell, use np.add.reduceat and np.take.

- At the CellCenter location, create a new FlowSolution_t node called "Centers."

    - Enter the calculated centroids in the "CCX," "CCY," and "CCZ" fields.

    - Connect this node to the zone.

In [6]:
def compute_cc_seq(tree):
    zone = PT.get_node_from_label(tree, 'Zone_t')

    cx, cy, cz = PT.Zone.coordinates(zone)
    connec = PT.get_node_from_name(zone, 'ElementConnectivity')[1]

    n_elem = connec.size // 4
    connec_idx = 4*np.arange(n_elem+1)

    mean_x = np.add.reduceat(np.take(cx, connec-1), connec_idx[:-1]) / 4
    mean_y = np.add.reduceat(np.take(cy, connec-1), connec_idx[:-1]) / 4
    mean_z = np.add.reduceat(np.take(cz, connec-1), connec_idx[:-1]) / 4

    PT.new_FlowSolution('Centers',
                        loc='CellCenter',
                        fields={'CCX' : mean_x, 'CCY' : mean_y, 'CCZ' : mean_z},
                        parent=zone)

## Step 4 -- Calcul cells's center withn a distributed CGNS tree

- To calculate cell centers (centroids) on a distributed CGNS tree, define the function compute_cc_dist(dist_tree, comm).

- Within the function:

    - Use PT.get_node_from_label(dist_tree, 'Zone_t') to retrieve the zone node from the distributed tree.

    - Use PT.Zone.coordinates(zone) to extract the distributed node coordinates cx, cy, and cz.

    - Use PT.get_node_from_name(zone, 'ElementConnectivity')[1] to obtain the element connectivity array.

- Use PT.maia.getDistribution(zone, 'Vertex')[1] to obtain the vertex distribution.

- Using the connectivity array, the MPI communicator comm, and the vertex distribution, create a DIndexer instance.

- Assume that every cell is a quad with four nodes.

    - Construct the connectivity index array by calculating the number of elements: connec_idx = 4 * np.arange(dn_elem+1).

- To remap distributed coordinates to a local view, utilize the indexer.take() method:

    - Use np.add.reduceat(...) / 4 to calculate the mean coordinates (mean_x, mean_y, and mean_z) for each cell.

- Make a brand-new FlowSolution_t node called 'Centers' at the CellCenter location.

    - Add the computed centroid coordinates as fields 'CCX', 'CCY', and 'CCZ'.

    - Attach this solution node to the zone.

In [7]:
def compute_cc_dist(dist_tree, comm):
    zone = PT.get_node_from_label(dist_tree, 'Zone_t')

    cx, cy, cz = PT.Zone.coordinates(zone)
    connec = PT.get_node_from_name(zone, 'ElementConnectivity')[1]

    vtx_distri = PT.maia.get_Distribution(zone, 'Vertex')[1]
    indexer = DIndexer(vtx_distri, connec, comm)
    

    dn_elem = connec.size // 4
    connec_idx = 4*np.arange(dn_elem+1)

    mean_x = np.add.reduceat(indexer.take(cx), connec_idx[:-1]) / 4
    mean_y = np.add.reduceat(indexer.take(cy), connec_idx[:-1]) / 4
    mean_z = np.add.reduceat(indexer.take(cz), connec_idx[:-1]) / 4
    
    PT.new_FlowSolution('Centers',
                        loc='CellCenter',
                        fields={'CCX' : mean_x, 'CCY' : mean_y, 'CCZ' : mean_z},
                        parent=zone)


## Step 5 -- use bigger meshes

NB : you can try with bigger meshes, first you need to generate it using
dist_tree = maia.factory.generate_dist_block(101, 'TETRA_4', comm)
maia.io.dist_tree_to_file(dist_tree, 'tetra100.hdf', comm)

Use the file 'tetra10.hdt' to calculate cells center with the function compute_cc_seq, then save the resulting tree to a file named sol.hdf. 

In [8]:
_generate_case()

In [9]:
# Sequential
if comm.rank == 0:
    tree = maia.io.read_tree(FILENAME)
    compute_cc_seq(tree)
    maia.io.write_tree(tree, 'sol.hdf')
    PT.print_tree(tree)

CGNSTree CGNSTree_t 
├───CGNSLibraryVersion CGNSLibraryVersion_t R4 [4.2]
└───Base CGNSBase_t I4 [3 3]
    └───zone Zone_t I4 [[1331 5000    0]]
        ├───ZoneType ZoneType_t "Unstructured"
        ├───GridCoordinates GridCoordinates_t 
        │   ├───CoordinateX DataArray_t R8 (1331,)
        │   ├───CoordinateY DataArray_t R8 (1331,)
        │   └───CoordinateZ DataArray_t R8 (1331,)
        ├───TETRA_4.0 Elements_t I4 [10  0]
        │   ├───ElementRange IndexRange_t I4 [   1 5000]
        │   └───ElementConnectivity DataArray_t I4 (20000,)
        ├───TRI_3.0 Elements_t I4 [5 0]
        │   ├───ElementRange IndexRange_t I4 [5001 6200]
        │   └───ElementConnectivity DataArray_t I4 (3600,)
        ├───ZoneBC ZoneBC_t 
        │   ├───Zmin BC_t "Null"
        │   │   ├───GridLocation GridLocation_t "FaceCenter"
        │   │   └───PointList IndexArray_t I4 (1, 200)
        │   ├───Zmax BC_t "Null"
        │   │   ├───GridLocation GridLocation_t "FaceCenter"
        │   │   └──

Now, we want to use the same file 'tetra10.hdf' with parallel partiotioned. 

In [10]:
tree = maia.io.file_to_dist_tree(FILENAME, comm)
ptree = maia.factory.partition_dist_tree(tree, comm)
compute_cc_seq(ptree)
maia.transfer.part_tree_to_dist_tree_all(tree, ptree, comm)
maia.io.dist_tree_to_file(tree, 'sol.hdf', comm)
PT.print_tree(tree)

Distributed read of file /home/jcoulet/Public/maia_training/MESHES/tetra10.hdf...
Read completed (0.06 s) -- Size of dist_tree for current rank is 143.7KiB (Σ=143.7KiB)
Partitioning tree of 1 initial block...
Partitioning completed (0.08 s) -- Nb of cells for current rank is 5.0K (Σ=5.0K)
Distributed write of a 262.2KiB dist_tree (Σ=262.2KiB)...
CGNSTree CGNSTree_t 
Write completed [sol.hdf] (0.53 s)
├───CGNSLibraryVersion CGNSLibraryVersion_t R4 [4.2]
└───Base CGNSBase_t I4 [3 3]
    └───zone Zone_t I4 [[1331 5000    0]]
        ├───ZoneType ZoneType_t "Unstructured"
        ├───GridCoordinates GridCoordinates_t 
        │   ├───CoordinateX DataArray_t R8 (1331,)
        │   ├───CoordinateY DataArray_t R8 (1331,)
        │   └───CoordinateZ DataArray_t R8 (1331,)
        ├───TETRA_4.0 Elements_t I4 [10  0]
        │   ├───ElementRange IndexRange_t I4 [   1 5000]
        │   ├───ElementConnectivity DataArray_t I4 (20000,)
        │   └───:CGNS#Distribution UserDefinedData_t 
        │ 

Now, we want to use the same file 'tetra10.hdf' with parallel distributed.

In [11]:
tree = maia.io.file_to_dist_tree(FILENAME, comm)
compute_cc_dist(tree, comm)
maia.io.dist_tree_to_file(tree, 'sol.hdf', comm)
PT.print_tree(tree)

Distributed read of file /home/jcoulet/Public/maia_training/MESHES/tetra10.hdf...
Read completed (0.07 s) -- Size of dist_tree for current rank is 143.7KiB (Σ=143.7KiB)
Distributed write of a 262.2KiB dist_tree (Σ=262.2KiB)...
CGNSTree CGNSTree_t 
Write completed [sol.hdf] (0.53 s)
├───CGNSLibraryVersion CGNSLibraryVersion_t R4 [4.2]
└───Base CGNSBase_t I4 [3 3]
    └───zone Zone_t I4 [[1331 5000    0]]
        ├───ZoneType ZoneType_t "Unstructured"
        ├───GridCoordinates GridCoordinates_t 
        │   ├───CoordinateX DataArray_t R8 (1331,)
        │   ├───CoordinateY DataArray_t R8 (1331,)
        │   └───CoordinateZ DataArray_t R8 (1331,)
        ├───TETRA_4.0 Elements_t I4 [10  0]
        │   ├───ElementRange IndexRange_t I4 [   1 5000]
        │   ├───ElementConnectivity DataArray_t I4 (20000,)
        │   └───:CGNS#Distribution UserDefinedData_t 
        │       └───Element DataArray_t I4 [   0 5000 5000]
        ├───TRI_3.0 Elements_t I4 [5 0]
        │   ├───ElementRange In